# Optimzing using Synthetic Q&A Data from Opik Traces

You will need:

1. A Comet account, for seeing Opik visualizations (free!) - [comet.com](https://comet.com)
2. An OpenAI account, for using an LLM
[platform.openai.com/settings/organization/api-keys](https://platform.openai.com/settings/organization/api-keys)

This example will use:

- [tinyqabenchmarkpp](https://pypi.org/project/tinyqabenchmarkpp/) to generate synthetic test dataset
- [opik-optimizer](https://pypi.org/project/opik-optimizer/) to optimize prompts


## Setup

This pip-install takes about a minute.

In [1]:
%pip install opik-optimizer tinyqabenchmarkpp --upgrade

You should consider upgrading via the '/Users/vincentkoc/GIT/_Perso/opik-main/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


This step configures the Opik library for your session. It will prompt for your Comet API key if not already set in your environment or through Opik's configuration.

In [2]:
import opik
opik.configure()

OPIK: Opik is already configured. You can check the settings by viewing the config file at /Users/vincentkoc/.opik.config


For this example, we'll use OpenAI models, so we need to set our OpenAI API key:

In [3]:
import os
import getpass
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Fetching Traces from Opik

Next up we will "fetch" existing traces of our AI application within Opik (we will use the demo project that ships with every Opik installation). 

This fetches traces from the demo project and formats them as a context string. Returns a formatted string with explanatory text and cleaned traces.

In [14]:
OPIK_PROJECT_NAME = "Demo chatbot 🤖"

# Will prompt for API key if not set
opik.configure()

# Fetch traces from the demo project
#
# Commented out the project name to
# fetch all traces across all projects
client = opik.Opik()
traces = client.search_traces(
    # project_name=OPIK_PROJECT_NAME,
    max_results=40
)

OPIK: Opik is already configured. You can check the settings by viewing the config file at /Users/vincentkoc/.opik.config


In [15]:
print(f"Found {len(traces)} traces")

Found 9 traces


We will define some helper functions to clean and traverse the traces as we don't wan to send noise to the LLM and break the input to the synthetic data generation step.

In [16]:
import re

def extract_text_from_dict(d: dict) -> list[str]:
    """
    Recursively extracts text from a dictionary.
    """
    texts = []
    for key, value in d.items():
        if isinstance(value, str):
            texts.append(value)
        elif isinstance(value, dict):
            texts.extend(extract_text_from_dict(value))
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, str):
                    texts.append(item)
                elif isinstance(item, dict):
                    texts.extend(extract_text_from_dict(item))
    return texts

def clean_text(text: str) -> str:
    """
    Cleans text by removing special characters and normalizing whitespace.
    """
    if not text:
        return ""
    # Replace special characters with spaces, but keep basic punctuation
    text = re.sub(r'[^\w\s.,!?;:\'"-]', ' ', text)
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    # Remove any leading/trailing whitespace
    text = text.strip()
    return text

We are now ready to extract and clean the text from the traces.

In [17]:
# Extract and clean text from traces
cleaned_texts = []
for i, trace in enumerate(traces):
    # Extract from input
    if trace.input:
        if isinstance(trace.input, dict):
            texts = extract_text_from_dict(trace.input)
            for text in texts:
                cleaned = clean_text(text)
                if cleaned:
                    cleaned_texts.append(cleaned)
        elif isinstance(trace.input, str):
            cleaned = clean_text(trace.input)
            if cleaned:
                cleaned_texts.append(cleaned)
    
    # Extract from output
    if trace.output:
        if isinstance(trace.output, dict):
            texts = extract_text_from_dict(trace.output)
            for text in texts:
                cleaned = clean_text(text)
                if cleaned:
                    cleaned_texts.append(cleaned)
        elif isinstance(trace.output, str):
            cleaned = clean_text(trace.output)
            if cleaned:
                cleaned_texts.append(cleaned)
    
    # Extract from metadata if it exists
    if trace.metadata:
        if isinstance(trace.metadata, dict):
            texts = extract_text_from_dict(trace.metadata)
            for text in texts:
                cleaned = clean_text(text)
                if cleaned:
                    cleaned_texts.append(cleaned)

if not cleaned_texts:
    print("Debug: No text content found in traces. Here's what we got:")
    for i, trace in enumerate(traces[:5]):  # Show first 5 traces for debugging
        print(f"\nTrace {i}:")
        print(f"Input: {trace.input}")
        print(f"Output: {trace.output}")
        print(f"Metadata: {trace.metadata}")
    raise ValueError("No valid text content found in traces")


Let's quickly inspect the traces we have.

In [21]:
display(traces[0])

TracePublic(id='019644b7-7c5b-7c7f-8587-db780693d76b', project_id='0194fe4e-2e8c-77c0-a863-1621fb52d257', name='levenshtein_ratio_metric', start_time=datetime.datetime(2025, 4, 17, 17, 6, 37, 784083, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2025, 4, 17, 17, 6, 37, 784165, tzinfo=TzInfo(UTC)), input={'output': 'Paddington and Washington have a total of 320 goats.', 'reference': 'If Washington has 140 goats, Washington has 140+40 = <<140+40=180>>180 goats.\nIn total, they have 140+180 = <<140+180=320>>320 goats\n#### 320', 'ignored_kwargs': {'input': '', 'expected_output': 'If Washington has 140 goats, Washington has 140+40 = <<140+40=180>>180 goats.\nIn total, they have 140+180 = <<140+180=320>>320 goats\n#### 320', 'context': ''}}, output={'output': {'name': 'levenshtein_ratio_metric', 'value': 0.43523316062176165, 'reason': None, 'metadata': None, 'scoring_failed': False}}, metadata=None, tags=None, error_info=None, usage=None, created_at=datetime.datetime(2025, 4, 17, 17, 6, 3

Remove any duplicates while preserving the order

In [18]:
# Remove duplicates while preserving order
seen = set()
unique_texts = []
for text in cleaned_texts:
    if text not in seen:
        seen.add(text)
        unique_texts.append(text)

Now we need to create the `context` to pass to the synthetic data generation step

In [22]:
context = f"""
This is a collection of AI/LLM conversation traces from
a given Comet Opik observability project. The following
text contains various interactions and responses that
can be used to generate relevant questions and answers.
<input>
{chr(10).join(unique_texts)}
</input>
"""

In [30]:
print(f"Found and cleaned {len(unique_texts)} unique text segments from traces")
print(f"Total context length: {len(context)} characters")

Found and cleaned 124 unique text segments from traces
Total context length: 6214 characters


## Generating Synthethic Data

We are now ready to generate the synthethic data using `tinyqabenchmarkpp`

In [19]:
# Model for tinyqabenchmarkpp
TQB_GENERATOR_MODEL = "openai/gpt-4o-mini" 

# Number of questions to generate
TQB_NUM_QUESTIONS = 20

# Languages to generate questions in
TQB_LANGUAGES = "en"

# Categories to generate questions in
TQB_CATEGORIES = "use context provided and elaborate on it to generate a more detailed answers"

# Difficulty of the questions to generate
TQB_DIFFICULTY = "medium"

In [23]:
# Command to generate the synthetic data
command = [
    "python", "-m", "tinyqabenchmarkpp.generate",
    "--num", str(TQB_NUM_QUESTIONS),
    "--languages", TQB_LANGUAGES,
    "--categories", TQB_CATEGORIES,
    "--difficulty", TQB_DIFFICULTY,
    "--model", TQB_GENERATOR_MODEL,
    "--str-output",
    "--context", context
]

Now we run the synthetic data generation step, please be patient as the language model is called.

In [ ]:
# Use a subprocess to run the command
import subprocess
process = subprocess.run(command, capture_output=True, text=True, check=True)

if process.stderr:
    # Print the errors
    print("tinyqabenchmarkpp errors:")
    print(process.stderr)
else:
    # Print the output
    print("Synthetic data generated successfully")
    print(process.stdout)

Synthetic data generated successfully
{"text": "How does Opik ensure continuous improvement in LLM evaluations?", "label": "Opik", "context": "Opik is designed to empower continuous improvement through community-driven innovation.", "tags": {"category": "AI", "difficulty": "medium"}, "sha256": "1796defe2059d05d10be5b3550b489102a92dcf41ac65817aa26eaef1de2b0ce", "id": "7c2c211c", "lang": "en"}
{"text": "What specific features does the Opik platform provide for monitoring production applications?", "label": "Opik platform", "context": "Opik provides features for monitoring production applications, particularly LLM applications.", "tags": {"category": "AI", "difficulty": "medium"}, "sha256": "db3752ffb5eb8e15af0a28ffc5374b65618707a3720e2b441a4b29a5ca3bbd91", "id": "09c1a34a", "lang": "en"}
{"text": "How can users utilize the Opik dashboard to track their LLM performance?", "label": "Opik dashboard", "context": "Users can review their feedback scores, trace count, and tokens over time using

## Store New Dataset in Opik
We can use the Opik SDK to push this dataset to Opik

In [37]:
generated_data = process.stdout

Helper function to process the JSONL response and push to Opik
Once we have defined we will be able to run this

In [42]:
import json

def load_synthetic_data_to_opik(data_str):
    """Load JSONL synthetic data into Opik as a dataset."""
    items = []
    for line in data_str.strip().split('\n'):
        try:
            data = json.loads(line)
            if not isinstance(data, dict):
                continue
            item = {
                "question": data.get("text"),
                "answer": data.get("label"),
                "generated_context": data.get("context"),
                "category": data.get("tags", {}).get("category"),
                "difficulty": data.get("tags", {}).get("difficulty")
            }
            if item["question"] and item["answer"]:
                items.append(item)
        except Exception:
            continue

    if not items:
        print("No valid items found.")
        return None

    dataset_name = f"demo-tinyqab-dataset-{TQB_CATEGORIES.replace(',', '_')}-{TQB_NUM_QUESTIONS}"
    dataset_name = "".join(c if c.isalnum() or c in ['-', '_'] else '_' for c in dataset_name)

    opik_client = opik.Opik()
    dataset = opik_client.create_dataset(
        name=dataset_name,
        description=f"Synthetic QA from tinyqabenchmarkpp for {TQB_CATEGORIES}"
    )
    dataset.insert(items)
    print(f"Opik Dataset '{dataset.name}' created with ID: {dataset.id}")
    return dataset

Push the data to Opik using helper function

In [43]:
opik_synthetic_dataset = load_synthetic_data_to_opik(generated_data)
if not opik_synthetic_dataset:
    print("Failed to load synthetic data into Opik. Exiting.")

OPIK: Created a "demo-tinyqab-dataset-use_context_provided_and_elaborate_on_it_to_generate_a_more_detailed_answers-20" dataset at https://www.comet.com/opik/api/v1/session/redirect/datasets/?dataset_id=0196e9b9-73a7-751a-8d6f-77232aae8bb2&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


Opik Dataset 'demo-tinyqab-dataset-use_context_provided_and_elaborate_on_it_to_generate_a_more_detailed_answers-20' created with ID: 0196e9b9-73a7-751a-8d6f-77232aae8bb2


## Agent Optimization Using Synthetic Data

Lets import the required packages for the Opik Agent Optimizer SDK

In [37]:
from opik_optimizer import (
    MetaPromptOptimizer,
    MetricConfig,
    TaskConfig,
    from_dataset_field,
    from_llm_response_text,
)
from opik.evaluation.metrics import LevenshteinRatio

We need to setup some intputs to our optimizer such as our starting prompt and some other configuration items.

In [25]:
# Initial prompt for the optimizer
OPTIMIZER_INITIAL_PROMPT = "Be a helpful assistant."

# Model for Opik Agent Optimizer
OPTIMIZER_MODEL = "openai/gpt-4o-mini"

# Population size for the optimizer
# Reduced for quicker demo
OPTIMIZER_POPULATION_SIZE = 5 

# Number of generations for the optimizer
# Reduced for quicker demo
OPTIMIZER_NUM_GENERATIONS = 2

# Number of samples from dataset for optimization eval
OPTIMIZER_N_SAMPLES_OPTIMIZATION = 10

Now we can setup the metric configuration used for the evaluation, as well as he task_config for passing in the dataset headings and initial prompt.

We finally pass this to our optimizer to set this up. We are opting to use the `MetaPromptOptimizer` optimizer in the SDK.

In [38]:
# Metric Configuration
metric_config = MetricConfig(
    metric=LevenshteinRatio(project_name=OPIK_PROJECT_NAME),
    inputs={
        "output": from_llm_response_text(),
        "reference": from_dataset_field(name="answer"),
    },
)

# Task Configuration
task_config = TaskConfig(
    instruction_prompt=OPTIMIZER_INITIAL_PROMPT,
    input_dataset_fields=["question", "generated_context"],
    output_dataset_field="answer",
)

# Initialize the optimizer
optimizer = MetaPromptOptimizer(
    model=OPTIMIZER_MODEL,
    project_name=OPIK_PROJECT_NAME,
    population_size=OPTIMIZER_POPULATION_SIZE,
    num_generations=OPTIMIZER_NUM_GENERATIONS,
    enable_moo=False,
    enable_llm_crossover=True,
    infer_output_style=True,
    verbose=1,
)

Now we can run the optimizer on the dataset and initial starting prompt to find the best prompt based on our synthetic data.

In [ ]:
# Commented out if you wan to pull the dataset from Opik without having
# to generate the synthetic data again

# import opik
# opik_client = opik.Opik()
# opik_synthetic_dataset = opik_client.get_or_create_dataset("demo-tinyqab-dataset-use_context_provided_and_elaborate_on_it_to_generate_a_more_detailed_answers-20")

In [39]:
# Run the optimizer
result = optimizer.optimize_prompt(
    dataset=opik_synthetic_dataset,
    metric_config=metric_config,
    task_config=task_config,
    n_samples=OPTIMIZER_N_SAMPLES_OPTIMIZATION,
)

Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

╭─ demo-tinyqab-dataset-use_context_provided_and_elaborate_on_it_to_generate_a_more_detailed_answers-20 (10 sampl─╮
│                                                                                                                 │
│ Total time:        00:00:00                                                                                     │
│ Number of samples: 10                                                                                           │
│                                                                                                                 │
│ levenshtein_ratio_metric: 0.0225 (avg)                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=709570;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196e9d1-d6a7-7e08-a494-caa46bcb73aa&dataset_id=0196e9b9-73a7-751a-8d6f-77232aae8bb2&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

Optimizing Prompt:   0%|                    | 0/3 [00:00<?, ?round/s, best_score=0.0225, llm_calls=0] | 0/3 [0…

...

View the results ]8;id=379580;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196e9d3-47ca-73b2-b532-6206ba088459&dataset_id=0196e9b9-73a7-751a-8d6f-77232aae8bb2&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

╭─ demo-tinyqab-dataset-use_context_provided_and_elaborate_on_it_to_generate_a_more_detailed_answers-20 (10 sampl─╮
│                                                                                                                 │
│ Total time:        00:00:00                                                                                     │
│ Number of samples: 10                                                                                           │
│                                                                                                                 │
│ levenshtein_ratio_metric: 0.6542 (avg)                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=534277;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196e9d3-4d60-7b53-bd38-f4accc8e64d2&dataset_id=0196e9b9-73a7-751a-8d6f-77232aae8bb2&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

### Optimization process finished
We can output our results to show

In [41]:
result.display()

╔═════════════════════════════════════════════ Optimization Complete ═════════════════════════════════════════════╗
║                                                                                                                 ║
║ Optimizer:         Optimizer                                                                                    ║
║ Model Used:        openai/gpt-4o-mini (Temp: N/A)                                                               ║
║ Metric Evaluated:  levenshtein_ratio_metric                                                                     ║
║ Initial Score:     0.0225                                                                                       ║
║ Final Best Score:  0.6542                                                                                       ║
║ Total Improvement: 2812.63%                                                                                     ║
║ Rounds Completed:  2                                                                                            ║
║ Stopped Early:     True                                                                                         ║
║                                                                                                                 ║
║                                                                                                                 ║
║ ╭─────────────────────────────────── Final Optimized Prompt (Instruction) ────────────────────────────────────╮ ║
║ │                                                                                                             │ ║
║ │  Analyze the provided question and context to identify one concise keyword or phrase that encapsulates the  │ ║
║ │  core insight. Your answer must be a term or phrase present in the context to maximize relevance.           │ ║
║ │                                                                                                             │ ║
║ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ ║
║                                                                                                                 ║
╚═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝

# Next Steps

You can try out other optimizers. More details can be found in the [Opik Agent Optimizer documentation](https://www.comet.com/docs/opik/agent_optimization/overview).